In [15]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.models import LinearAxis, Range1d
import numpy as np

In [13]:
learning_rate = 0.001
batch_size = 100
num_classes = 10
num_epochs = 5

DATA_PATH = r'\Users\Nick\Desktop\Learning ML\Tutorials\MNIST PyTorch\MNISTData'
MODEL_STORE_PATH = r'\Users\Nick\Desktop\Learning ML\Tutorials\MNIST PyTorch\MNISTData\Model'

In [16]:
# transforms to apply to the data
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=True, transform=trans, download=True)
test_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=False, transform=trans)

0it [00:00, ?it/s]

9920512it [00:02, 4668187.12it/s]                             


Extracting \Users\Nick\Desktop\Learning ML\Tutorials\MNIST PyTorch\MNISTData/MNIST/raw/train-images-idx3-ubyte.gz to \Users\Nick\Desktop\Learning ML\Tutorials\MNIST PyTorch\MNISTData/MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 160057.10it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

...

1654784it [00:01, 1344351.66it/s]                            


Extracting \Users\Nick\Desktop\Learning ML\Tutorials\MNIST PyTorch\MNISTData/MNIST/raw/t10k-images-idx3-ubyte.gz to \Users\Nick\Desktop\Learning ML\Tutorials\MNIST PyTorch\MNISTData/MNIST/raw


8192it [00:00, 55498.69it/s]            

Extracting \Users\Nick\Desktop\Learning ML\Tutorials\MNIST PyTorch\MNISTData/MNIST/raw/t10k-labels-idx1-ubyte.gz to \Users\Nick\Desktop\Learning ML\Tutorials\MNIST PyTorch\MNISTData/MNIST/raw
Processing...


Done!


In [17]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: \Users\Nick\Desktop\Learning ML\Tutorials\MNIST PyTorch\MNISTData
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [18]:
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False)

In [29]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        # The input dimensions must be implied - we only specify the incoming
        # and outgoing number of channels in each layer
        
        # Each layer has a convolution, a ReLU activation, and a pooling layer
        self.layer1 = nn.Sequential(
            
            # 1 is the number of channels in the input, and 32 is the number of channels in the output
            # ** Question: how are the 32 filters created, specifically? Are they
            # pre-programmed in torch?
            nn.Conv2d(1, 32, kernel_size = 5, stride = 1, padding = 2),
            
            # ReLU keeps learned parameters from getting stuck near 0, 
            # ob lowing up to infinity
            nn.ReLU(),
            
            # pooling reduces the number of parameters we need to learn
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size = 5, stride = 1, padding = 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        # add a dropout layer to prevent over-fitting
        self.drop_out = nn.Dropout()
        
        # Fully-connected layers
        self.fc1 = nn.Linear(7*7*64, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)

        # flatten the output
        out = out.reshape(out.size(0), -1)

        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)

        return out
        

In [34]:
model = ConvNet()

In [35]:
# When to use one loss vs another?
# When to use one optimizer vs another?

# Use cross-entropy loss for classification problems
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [36]:
# Train the model
total_step = len(train_loader)
loss_list = []
acc_list = []

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
        
        # do backprop and step forward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/5], Step [100/600], Loss: 0.1586, Accuracy: 95.00%
Epoch [1/5], Step [200/600], Loss: 0.2013, Accuracy: 94.00%
Epoch [1/5], Step [300/600], Loss: 0.2614, Accuracy: 93.00%
Epoch [1/5], Step [400/600], Loss: 0.0533, Accuracy: 99.00%
Epoch [1/5], Step [500/600], Loss: 0.1639, Accuracy: 93.00%
Epoch [1/5], Step [600/600], Loss: 0.2160, Accuracy: 95.00%
Epoch [2/5], Step [100/600], Loss: 0.1163, Accuracy: 95.00%
Epoch [2/5], Step [200/600], Loss: 0.0944, Accuracy: 97.00%
Epoch [2/5], Step [300/600], Loss: 0.0424, Accuracy: 98.00%
Epoch [2/5], Step [400/600], Loss: 0.1600, Accuracy: 98.00%
Epoch [2/5], Step [500/600], Loss: 0.1934, Accuracy: 95.00%
Epoch [2/5], Step [600/600], Loss: 0.0837, Accuracy: 96.00%
Epoch [3/5], Step [100/600], Loss: 0.0747, Accuracy: 96.00%
Epoch [3/5], Step [200/600], Loss: 0.0896, Accuracy: 97.00%
Epoch [3/5], Step [300/600], Loss: 0.0598, Accuracy: 98.00%
Epoch [3/5], Step [400/600], Loss: 0.0707, Accuracy: 97.00%
Epoch [3/5], Step [500/600], Loss: 0.064

In [75]:
loss_list

[2.3085732460021973,
 4.266198635101318,
 2.3248226642608643,
 2.1178548336029053,
 2.1375582218170166,
 2.0911998748779297,
 1.9743396043777466,
 1.866123914718628,
 1.8232954740524292,
 1.698809027671814,
 1.50838303565979,
 1.3309123516082764,
 1.2584373950958252,
 1.0189917087554932,
 1.0921680927276611,
 0.8304089903831482,
 0.8815823197364807,
 0.8123769164085388,
 0.6732941269874573,
 0.678486168384552,
 0.8088133931159973,
 0.5870787501335144,
 0.8281004428863525,
 0.5033937692642212,
 0.5520946383476257,
 0.9426037073135376,
 0.48066917061805725,
 0.5877964496612549,
 0.5544633269309998,
 0.5281856656074524,
 0.4316374957561493,
 0.365474671125412,
 0.3871639668941498,
 0.3633579909801483,
 0.3447851240634918,
 0.5049936175346375,
 0.3858660161495209,
 0.2695232629776001,
 0.39952027797698975,
 0.24616193771362305,
 0.33683040738105774,
 0.27180150151252747,
 0.3865392804145813,
 0.4790905714035034,
 0.28266769647598267,
 0.16707399487495422,
 0.29590731859207153,
 0.324713855

In [37]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format((correct / total) * 100))


Test Accuracy of the model on the 10000 test images: 99.05000000000001 %


In [38]:
torch.save(model.state_dict(), MODEL_STORE_PATH + 'conv_net_model.ckpt')

p = figure(y_axis_label='Loss', width=850, y_range=(0, 1), title='PyTorch ConvNet results')
p.extra_y_ranges = {'Accuracy': Range1d(start=0, end=100)}
p.add_layout(LinearAxis(y_range_name='Accuracy', axis_label='Accuracy (%)'), 'right')
p.line(np.arange(len(loss_list)), loss_list)
p.line(np.arange(len(loss_list)), np.array(acc_list) * 100, y_range_name='Accuracy', color='red')
show(p)

In [70]:
# Get one example from the test dataset
data, target = [ x[0] for x in iter(test_loader).next() ]

In [71]:
target

tensor(7)

In [72]:
data = data.view(-1, 1, 28, 28)  

In [73]:
#data, target = Variable(data, volatile=True), Variable(target)
#data = data.view(-1, 28 * 28)
net_out = model(data)
net_out.argmax()

tensor(7)

In [74]:
torch.max(net_out.data, 1)

torch.return_types.max(
values=tensor([23.8073]),
indices=tensor([7]))